![A soccer pitch for an international match.](soccer-pitch.jpg)

You're working as a sports journalist at a major online sports media company, specializing in soccer analysis and reporting. You've been watching both men's and women's international soccer matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official `FIFA World Cup` matches (not including qualifiers) since `2002-01-01`.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: `women_results.csv` and `men_results.csv`.

The question you are trying to determine the answer to is:

> Are more goals scored in women's international soccer matches than men's?

You assume a **10% significance level**, and use the following null and alternative hypotheses:

$H_0$ : The mean number of goals scored in women's international soccer matches is the same as men's.

$H_A$ : The mean number of goals scored in women's international soccer matches is greater than men's.

In [10]:
# Start your code here!
import pandas as pd
men_df = pd.read_csv("men_results.csv")

men_df = men_df[men_df["date"]<"2002-01-01"]
men_df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 24863 entries, 0 to 24862
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  24863 non-null  int64 
 1   date        24863 non-null  object
 2   home_team   24863 non-null  object
 3   away_team   24863 non-null  object
 4   home_score  24863 non-null  int64 
 5   away_score  24863 non-null  int64 
 6   tournament  24863 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.5+ MB


In [11]:
women_df = pd.read_csv("women_results.csv")
women_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 0 to 4883
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4884 non-null   int64 
 1   date        4884 non-null   object
 2   home_team   4884 non-null   object
 3   away_team   4884 non-null   object
 4   home_score  4884 non-null   int64 
 5   away_score  4884 non-null   int64 
 6   tournament  4884 non-null   object
dtypes: int64(3), object(4)
memory usage: 267.2+ KB


In [12]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu, shapiro
import warnings
warnings.filterwarnings('ignore')

# Load data
men_df = pd.read_csv("men_results.csv")
women_df = pd.read_csv("women_results.csv")

# Convert date to datetime and filter since 2002-01-01
men_df['date'] = pd.to_datetime(men_df['date'])
women_df['date'] = pd.to_datetime(women_df['date'])

men_df = men_df[men_df['date'] >= '2002-01-01']
women_df = women_df[women_df['date'] >= '2002-01-01']

# Filter for FIFA World Cup only (exclude qualifiers)
men_world_cup = men_df[men_df['tournament'] == 'FIFA World Cup']
women_world_cup = women_df[women_df['tournament'] == 'FIFA World Cup']

# Calculate total goals per match
men_world_cup['total_goals'] = men_world_cup['home_score'] + men_world_cup['away_score']
women_world_cup['total_goals'] = women_world_cup['home_score'] + women_world_cup['away_score']

men_goals = men_world_cup['total_goals']
women_goals = women_world_cup['total_goals']

# === Step 1: Descriptive Statistics ===
men_mean = men_goals.mean()
women_mean = women_goals.mean()
men_n = len(men_goals)
women_n = len(women_goals)

print(f"Men's FIFA World Cup matches (2002+): {men_n}")
print(f"Women's FIFA World Cup matches (2002+): {women_n}")
print(f"Mean goals per match (Men): {men_mean:.2f}")
print(f"Mean goals per match (Women): {women_mean:.2f}")

# === Step 2: Check Normality (Shapiro-Wilk) ===
_, men_p_norm = shapiro(men_goals)
_, women_p_norm = shapiro(women_goals)

print(f"\nShapiro-Wilk p-value (Men): {men_p_norm:.6f}")
print(f"Shapiro-Wilk p-value (Women): {women_p_norm:.6f}")

if men_p_norm < 0.05 or women_p_norm < 0.05:
    print("At least one distribution is not normal → Use non-parametric test.")
else:
    print("Both distributions appear normal → Could use t-test.")

# === Step 3: Mann-Whitney U Test (one-sided: women > men) ===
stat, p_two_sided = mannwhitneyu(women_goals, men_goals, alternative='two-sided')
p_val = p_two_sided / 2  # Convert to one-sided (greater)

print(f"\nMann-Whitney U Test:")
print(f"U statistic: {stat:.2f}")
print(f"Two-sided p-value: {p_two_sided:.6f}")
print(f"One-sided p-value (women > men): {p_val:.6f}")

# === Step 4: Decision at α = 0.10 ===
alpha = 0.10
if p_val < alpha:
    result = "reject"
    print(f"p-value ({p_val:.6f}) < α ({alpha}) → Reject H0")
else:
    result = "fail to reject"
    print(f"p-value ({p_val:.6f}) ≥ α ({alpha}) → Fail to reject H0")

# === Final Result Dictionary ===
result_dict = {"p_val": round(p_val, 4), "result": result}

print("\n" + "="*50)
print("FINAL RESULT:")
print(result_dict)

Men's FIFA World Cup matches (2002+): 384
Women's FIFA World Cup matches (2002+): 200
Mean goals per match (Men): 2.51
Mean goals per match (Women): 2.98

Shapiro-Wilk p-value (Men): 0.000000
Shapiro-Wilk p-value (Women): 0.000000
At least one distribution is not normal → Use non-parametric test.

Mann-Whitney U Test:
U statistic: 43273.00
Two-sided p-value: 0.010213
One-sided p-value (women > men): 0.005107
p-value (0.005107) < α (0.1) → Reject H0

FINAL RESULT:
{'p_val': 0.0051, 'result': 'reject'}
